<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">

### Movie_Recomendation system can be based on basis of three types:-

1. Based on content watched by user. (Content based)
2. Based on content watched by other similar user. (Collaborative filtering)
3. Above both combined together (Hybrid)

- Here we are going to implement content based Movie recommendation system.
    
</div>

In [49]:
import numpy as np
import pandas as pd

In [50]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [51]:
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [52]:
print(credits.shape)
credits.head(1)

(4803, 4)


,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [53]:
movies = movies.merge(credits, on='title')
print(movies.shape)
movies.head(1)

(4809, 23)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [54]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">

### Selection of Important Parameters from the Dataset ( **Preprocessing** )

After merging both datasets to make them easier to use, we will now proceed to select important parameters from the 23 available columns, keeping only those that are useful for our analysis. The following parameters will be retained:

1. **genres**: The genre of the movie, which helps in categorizing films based on type.
2. **id**: A unique identifier for each movie, essential for indexing and merging datasets.
3. **title**: The title of the movie, useful for identifying the films.
4. **overview**: A brief description of the movie, which can be useful for understanding the storyline.
5. **cast**: The list of actors featured in the movie, which is important for analyzing star power or casting trends.
6. **crew**: Information about the crew, including directors and writers, critical for production insights.
7. **keywords**: Tags or keywords related to the movie, helpful for understanding themes or recurring elements.

Parameters to consider later
- **revenue**: Although not used for now, it could be valuable for financial analysis at a later stage.
- **popularity**: Not being used for now but may be useful for analyzing audience reception in future studies.
- **release date**: Not used now but important for targeting era based audience.

Parameters we have **neglected**:
- **language**: Since the majority of films are in English, it won't provide significant insights.
- **tagline**: It can be misleading or inconsistent, and we've already included the `overview` field, which gives more context about the movie.

By narrowing down to these parameters, we ensure that we are focusing on the most relevant data for our analysis, without introducing unnecessary complexity.

</div>

In [55]:
movies = movies[['id','title','overview', 'genres','keywords','cast','crew']]
movies.head

<bound method NDFrame.head of           id                                     title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4804    9367                               El Mariachi   
4805   72766                                 Newlyweds   
4806  231617                 Signed, Sealed, Delivered   
4807  126186                          Shanghai Calling   
4808   25975                         My Date with Drew   

                                               overview  \
0     In the 22nd century, a paraplegic Marine is di...   
1     Captain Barbossa, long believed to be dead, ha...   
2     A cryptic message from Bond’s past sends him o...   
3     Following the death of District

<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">

### Reformatting the table data ( **Preprocessing** )
- Column of table will contain only three parameters - id, title, tag.
- For tag we will take important info from all rest column and will form it.
- But first need to process data of each column so that only important data remains.
    
</div>

In [56]:
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [57]:
# Since only 3 value in overview are null neglecting them will not affect much
movies.dropna(inplace=True)

In [58]:
movies.duplicated().sum()

np.int64(0)

In [59]:
movies.iloc[0].genres 

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [60]:
import ast

def convert(obj):
    l=[];
    
    # obj is string of list which cannot be used directly first converting it to list
    ob = ast.literal_eval(obj)
    for i in ob:
        l.append(i["name"])
    return l

In [61]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [62]:
def convert3(obj):
    l=[];
    
    # obj is string of list which cannot be used directly first converting it to list
    ob = ast.literal_eval(obj)
    
    cnt=0
    for i in ob:
        if cnt<3:
            l.append(i["name"])
            cnt += 1
        else:
            break
    return l

In [63]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [64]:
def find_director(obj):
    ob = ast.literal_eval(obj)
    l = [];
    
    for i in ob:
        if i["job"] == "Director":
            l.append(i["name"])
            
    return l

In [65]:
movies['crew'] = movies['crew'].apply(find_director)
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [66]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [67]:
import string
import ast

stop_words = {  
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',  
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself',  
    'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them',
    'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',  
    'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been',  
    'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',  
    'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',  
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',  
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',  
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again',   
    'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why',   
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',   
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than',   
    'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should',   
    'now' 
}

def process_text(data):
    l = []
    words = data.split()
    
    for word in words:
        word = word.translate(str.maketrans('', '', string.punctuation))

        if word.lower() not in stop_words:
            l.append(word.lower())
    
    return l


In [68]:
movies['overview'] = movies['overview'].apply(process_text)
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[22nd, century, paraplegic, marine, dispatched...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[captain, barbossa, long, believed, dead, come...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[cryptic, message, bond’s, past, sends, trail,...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[following, death, district, attorney, harvey,...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[john, carter, warweary, former, military, cap...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [69]:
# Removing spaces between name in keywords, cast, crew, genres to avoid any error
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[22nd, century, paraplegic, marine, dispatched...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[captain, barbossa, long, believed, dead, come...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[cryptic, message, bond’s, past, sends, trail,...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[following, death, district, attorney, harvey,...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[john, carter, warweary, former, military, cap...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [70]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[22nd, century, paraplegic, marine, dispatched...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[22nd, century, paraplegic, marine, dispatched..."
1,285,Pirates of the Caribbean: At World's End,"[captain, barbossa, long, believed, dead, come...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[captain, barbossa, long, believed, dead, come..."
2,206647,Spectre,"[cryptic, message, bond’s, past, sends, trail,...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[cryptic, message, bond’s, past, sends, trail,..."
3,49026,The Dark Knight Rises,"[following, death, district, attorney, harvey,...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[following, death, district, attorney, harvey,..."
4,49529,John Carter,"[john, carter, warweary, former, military, cap...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[john, carter, warweary, former, military, cap..."


In [71]:
movies = movies[['id','title','tags']]
movies.head()

,id,title,tags
0,19995,Avatar,"[22nd, century, paraplegic, marine, dispatched..."
1,285,Pirates of the Caribbean: At World's End,"[captain, barbossa, long, believed, dead, come..."
2,206647,Spectre,"[cryptic, message, bond’s, past, sends, trail,..."
3,49026,The Dark Knight Rises,"[following, death, district, attorney, harvey,..."
4,49529,John Carter,"[john, carter, warweary, former, military, cap..."


In [72]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

C:\Users\Mukesh Choudhary\AppData\Local\Temp\ipykernel_19680\2153568569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))


In [73]:
movies.iloc[0].tags

'22nd century paraplegic marine dispatched moon pandora unique mission becomes torn following orders protecting alien civilization Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [74]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())
movies.head()

C:\Users\Mukesh Choudhary\AppData\Local\Temp\ipykernel_19680\2725882095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x: x.lower())


,id,title,tags
0,19995,Avatar,22nd century paraplegic marine dispatched moon...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed dead come back ...
2,206647,Spectre,cryptic message bond’s past sends trail uncove...
3,49026,The Dark Knight Rises,following death district attorney harvey dent ...
4,49529,John Carter,john carter warweary former military captain w...


<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">
    
### Issues in Tags
- (actor, actors), (activity, activities), (action, actions), etc. are considered different, but they should be treated as the same.
- To solve this, we will use the **NLTK** library, which will help in grouping these terms under one heading.

</div>

In [75]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [76]:
def stems(text):
    l=[]
    
    for i in text.split():
        l.append(ps.stem(i))
    
    return " ".join(l)

In [77]:
movies['tags'] = movies['tags'].apply(stems)

C:\Users\Mukesh Choudhary\AppData\Local\Temp\ipykernel_19680\2623043491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(stems)


<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">


### Vectorization of Tags

- **Step 1**: First, extract all tag words from all movie tags.
- **Step 2**: Sum the occurrence of each tag word. If the total number of unique words is, say, 100,000, select the top 5000 unique words (you can choose a different number) with the highest frequency.
- **Step 3**: Represent each movie as a vector based on the occurrence of these selected words. This results in a vector of shape (1, 5000) for each movie, and the entire dataset becomes of shape (number of movies, 5000).
- **Step 4**: To recommend a movie, find the closest match to the vector of a watched movie using vector similarity.

</div>

In [78]:
import sklearn 

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [79]:
vectors = cv.fit_transform(movies['tags']).toarray()
vectors.shape

(4806, 5000)

In [80]:
feature_names = cv.get_feature_names_out().tolist()
#print(feature_names)

<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">


### To calculate distance between two vectors
- We will not calculate the euclidean distance as it can cause error. 
- We will calculate cosine distance for checking similarity.

</div>

In [81]:
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
similarity = cosine_similarity(vectors)
similarity.shape

(4806, 4806)

<div style="border:1px solid #000; padding: 10px; background-color: #FF8C00; border-radius: 10px;">


### Main function to fetch similar movies to given movie
- Select index of movie from movies datframe
- Access distance vector of that movie with every other movie by similarity(index)
- Now use enumerate to store index along with similarity in distance, then make list
- Now sort the list on basis of similarity in distance hence finding index similar to earlier one

</div>

In [83]:
def recommend_movie(movie_name):
    ind = movies[movies['title'] == movie_name].index[0]
    distance = similarity[ind]
    movie_list = sorted(list(enumerate(distance)), reverse=True, key = lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(movies.iloc[i[0]].title)

In [87]:
print(movies['title'][1])
recommend_movie(movies['title'][1])

Pirates of the Caribbean: At World's End
Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
Life of Pi
20,000 Leagues Under the Sea


In [88]:
# picle for making website
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))

In [89]:
pickle.dump(movies.to_dict(), open('movie_dict.pkl','wb'))